## Looking at Some LSSTCam DRP subtractions

Notebook by Michael Wood-Vasey: <wmwv@pitt.edu>  
Started 2025-05-16 based on similar notebook for ComCam  

The first part of this notebook demonstrates how to look at sources from one image subtraction.  the first-round DIA results using the interactive firefly backend. It also provides guidance on obtaining clean DIA sources through flag selection and applying a signal-to-noise ratio cut. It serves as a starting point for those interested in evaluating the subtraction quality of the LSSTCam data.

The second part of this notebook looks at the DIAObject catalog more broadly (for the same region of sky) and displays the image stamps (either from the visit images or the difference images) for a selected DIAObject.

In [ ]:
import matplotlib.pyplot as plt

from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay

import lsst.geom

import lsst.afw.image
from lsst.afw.math import Warper, WarperConfig
import lsst.afw.table

from lsst.ip.diffim import AlardLuptonSubtractConfig, AlardLuptonSubtractTask
from lsst.ip.diffim import GetTemplateConfig, GetTemplateTask
from lsst.ip.diffim import DetectAndMeasureConfig, DetectAndMeasureTask

In [ ]:
# Choose backend
afwDisplay.setDefaultBackend("firefly")
# afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
# Load collection
repo = "embargo_new"
# https://ls.st/campaigns
# Follow to Intermittent Processing Campaigns for Cumulative DRP.
collection = "LSSTCam/runs/DRP/FL/w_2025_19/DM-50795"

instrument = "LSSTCam"

butler = Butler(repo, collections=collection)

name_skymap = "lsst_cells_v1"
skymap = butler.get("skyMap", skymap=name_skymap, collections="skymaps")

In [ ]:
butler.query_datasets("dia_source")

### Identify data_ids from campaign

Translations

New | Old
--- | ---
source | src
visit_image | calexp
template_detector | goodSeeingDiff_templateExp  
source_detector	 | 
source	  | 
difference_image_predetection | goodSeeingDiff_differenceTempExp  
difference_kernel | goodSeeingDiff_psfMatchKernel  
difference_kernel_sources | goodSeeingDiff_psfMatchSources  
template_matched | goodSeeingDiff_matchedExp  
dia_source_unstandardized | goodSeeingDiff_diaSrc  
difference_image | goodSeeingDiff_differenceExp  
dia_source_reliability | goodSeeingRealBogusSources  
dia_source_detector | goodSeeingDiff_diaSrcTable  
dia_source_visit | diaSourceTable  

In [ ]:
band = "i"

datasetRefs_dia = butler.query_datasets("difference_image", where=f"band='{band}'")

In [ ]:
verbose = False
if verbose:
    for dr in datasetRefs_dia:
        print(dr)

print(f"\nFound {len(datasetRefs_dia)} differenceExps")

In [ ]:
i = 40
data_id = datasetRefs_dia[i].dataId
print(data_id)

In [ ]:
data_id = {"instrument": "LSSTCam", "visit": 2025042400229, "detector": 4}

In [ ]:
diff = butler.get("difference_image", dataId=data_id)
template = butler.get("template_detector", dataId=data_id)
image = butler.get("visit_image", dataId=data_id)
image_background = butler.get("visit_image_background", dataId=data_id)

In [ ]:
source = butler.get("single_visit_star_footprints", dataId=data_id)
dia_source = butler.get("dia_source_unstandardized", dataId=data_id)
dia_source_detector = butler.get("dia_source_detector", dataId=data_id)  # SDM-ified table.  Does not containg sky sources

In [ ]:
afw_display = afwDisplay.Display(frame=1)

afw_display.setMaskTransparency(80)
# afw_display.scale("asinh", -20, 50)
afw_display.scale("linear", "zscale")

afw_display.mtv(template)

In [ ]:
afw_display = afwDisplay.Display(frame=2)

afw_display.setMaskTransparency(80)
# afw_display.scale("asinh", -2, 5)
afw_display.scale("linear", "zscale")

afw_display.mtv(image)

In [ ]:
afw_display = afwDisplay.Display(frame=3)

afw_display.setMaskTransparency(100)
afw_display.scale("linear", "zscale")
afw_display.mtv(diff)

Then go to the Firefly window and click the "link" button in the toolbar (between the book of mask plans and the full-screen diagonal arrow).  Select "Align and Lock Options" -> "by Pixel at Image Centers" and then things will move together when you pan around and zoom in and out.  I don't know how to do this programatically as a set in the Notebook.

In [ ]:
# We're mixing columsn from dia_src and dia_src_table
# This is dangerous.  I don't think there is a guarantee that the ordering is the same.
# But I want the flags from dia_src and the science flux from dia_src_table

"""
good = ~dia_src["slot_Shape_flag"] & \
    (dia_src["base_PsfFlux_instFlux"] / dia_src["base_PsfFlux_instFluxErr"] > snr_threshold) & \
    ~dia_src["base_PixelFlags_flag_edge"] & \
    ((dia_src_table["scienceFlux"] / dia_src_table["scienceFluxErr"]) < max_science_snr) & \
    ~dia_src_table["pixelFlags_streak"]
"""

def good_cat(cat, snr_threshold=7.5, max_science_snr=200):
    # If I were doing just dia_src_table, I'd do something like this:
    good = (cat["snr"] > snr_threshold) & \
        ~cat["shape_flag"] & \
        ~cat["pixelFlags_bad"] & \
        ~cat["pixelFlags_cr"] & \
        ((cat["scienceFlux"] / cat["scienceFluxErr"]) < max_science_snr)

    good_cat = cat[good].copy(deep=True)

    return good_cat

good_dia_source_detector = good_cat(dia_source_detector)

print(f"{len(good_dia_source_detector)} good DIA sources found out of {len(good_dia_source_detector)} detections.")

In [ ]:
good_dia_source_detector

In [ ]:
for (x, y) in zip(good_dia_source_detector["x"], good_dia_source_detector["y"]):
    afw_display.dot("o", x, y, size=20, ctype="green")

In [ ]:
good_dia_source_detector["scienceSnr"] = good_dia_source_detector["scienceFlux"] / good_dia_source_detector["scienceFluxErr"]

In [ ]:
# Candidates near the following x, y are real
# 839.87, 164.02  # likely nearby galaxy
# 1501, 3105  # likely nearby galaxy
# 3940, 1263  # likely nearby galaxy
#  157,  465  # no obvious galaxy nearby
# 1155, 3230  # likely nearby galaxy

In [ ]:
good_dia_source_detector[["ra", "dec", "x", "y", "snr", "scienceSnr"]]

In [ ]:
cand_idx = 16
src = good_dia_source_detector.loc[cand_idx] 
print(src["ra"], src["dec"])

In [ ]:
coord = lsst.geom.SpherePoint(src["ra"], src["dec"], units=lsst.geom.degrees)
tract = skymap.findTract(coord)
patch = tract.findPatch(coord)

In [ ]:
tract_data_id = {"tract": tract.tract_id, "patch": patch.getSequentialIndex(), "skymap": name_skymap}
dia_object = butler.get("dia_object", dataId=tract_data_id)
dia_object_patch = butler.get("dia_object_patch", dataId=tract_data_id)
dia_object_forced_source_patch = butler.get("dia_object_forced_source", dataId=tract_data_id)

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
skycoord = SkyCoord(src["ra"], src["dec"], unit=u.deg)

In [ ]:
dia_coord = SkyCoord(dia_object_patch["ra"], dia_object_patch["dec"], unit=u.deg)
idx, sep, _ = skycoord.match_to_catalog_sky(dia_coord)

In [ ]:
dia_object_patch.iloc[idx]

In [ ]:
dia_object_forced_source_patch[idx]

In [ ]:
print(len(dia_object_forced_source_patch))
print(len(dia_object_patch))
print(len(dia_object))

In [ ]:
dia_object_patch.nDiaSources[(5 < dia_object_patch.nDiaSources) & (dia_object_patch.nDiaSources < 13)]

In [ ]:
# dia_object_id = 69259886701051909  # Variable star
# dia_object_id = 69259886701051913  # Likely asteroid
# dia_object_id = 69259886701051905  # Likely asteroid
# dia_object_id = 69259886701051906  # Variable star
# dia_object_id = 69259886701051907  # Variable star
# dia_object_id = 69259886701051915  # Variable star
# dia_object_id = 69259886701052373  # Variable star
dia_object_id = 69259886701051927  # Variable star

dia_object_patch.loc[dia_object_id]

In [ ]:
foo = dia_object_forced_source_patch[(dia_object_forced_source_patch["diaObjectId"] == dia_object_id) & (dia_object_forced_source_patch["band"] == "i")]

In [ ]:
# plt.errorbar(foo["visit"], foo["psfFlux"], foo["psfFluxErr"], marker=".", linestyle="none", color="blue")
plt.errorbar(foo["visit"], foo["psfDiffFlux"], foo["psfDiffFluxErr"], marker=".", linestyle="none", color="orange")

In [ ]:
foo

Let's go through and look at the cutout stamps

In [ ]:
def cutout_image(butler, ra, dec, visit, detector, cutoutSideLength=51, dataset_type="visit_image", **kwargs):
    
    """
    Produce a cutout from an image at the given ra, dec position.

    If the cutout extends beyond the data array then an error will be triggered.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id of the image's visit
    detector: int
        Detector for the image
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.
    dataset_type: str
        dataset_type
        "visit_image", "template_matched", "difference_image", ...

    Returns
    -------
    MaskedImage
    """
    
    data_id = {"visit": visit, "detector": detector}    
    radec = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)
    cutoutSize = lsst.geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    wcs = butler.get(dataset_type+".wcs", **data_id)
    xy = lsst.geom.PointI(wcs.skyToPixel(radec))
    bbox = lsst.geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get(dataset_type, parameters=parameters, **data_id)

    return cutout_image

In [ ]:
afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
i = 0
cutout = cutout_image(butler, ra=foo["coord_ra"][i], dec=foo["coord_dec"][i], visit=foo["visit"][i], detector=foo["detector"][i])

In [ ]:
dataset_type = "difference_image"
# dataset_type = "visit_image"
for i, row in enumerate(foo):
    print(row["visit"])
    try:
        cutout = cutout_image(butler, ra=row["coord_ra"], dec=row["coord_dec"], visit=row["visit"], detector=row["detector"], dataset_type=dataset_type)
    except FileNotFoundError as e:
        print(e)
        continue
    except ValueError as e:
        print(e)
        continue
    except FitsError as e:
        print(e)
        continue

    afw_display = afwDisplay.Display(frame=i)
    afw_display.scale("linear", "zscale")
    afw_display.mtv(cutout)


In [ ]:
dia_object_patch.columns